In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
import time

In [2]:
#Ball tracker

In [8]:
#defining lower and upper boundaries of the "green" in the ball in HSV 
#color space
green_lower = (29,86,6)
green_upper = (64,255,255)
#list of tracked points
pts = deque(maxlen = 50)

cap = cv2.VideoCapture(0)

time.sleep(2.0)

while True:
    ret, frame = cap.read()
    #blur the frame
    blurred = cv2.GaussianBlur(frame,(5,5),0)
    #convert it to HSV color space
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    
    # construct a mask for the color "green", then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv,green_lower,green_upper)
    mask = cv2.erode(mask,None,iterations = 2)
    mask = cv2.dilate(mask,None,iterations = 2)
    #cv2.imshow('Frame', frame)
    #cv2.imshow('Mask',mask)
    
    #find contours in the mask and init. (x,y) center of the ball
    im2,cnts,heirarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #find largest contour in the mask
    center = None
    if len(cnts) > 0:
        c = max(cnts,key=cv2.contourArea)
        #Compute min. enclosing circle and centroid
        ((x,y),radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))
    
        # only proceed if the radius meets a minimum size
        # draw the circle and centroid on the frame,
        # then update the list of tracked points
        if radius > 3:
            cv2.circle(frame, (int(x),int(y)), int(radius),(0,255,255),2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
        
    #update the points queue
    pts.appendleft(center)
    
    #loop over the set of tracked points
    for i in range(1,len(pts)):
        #if either of the tracked points are None, ignore them
        if pts[i-1] is None or pts[i] is None:
            continue
            
        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        cv2.line(frame,pts[i-1],pts[i],(0,0,255),1)
        
    frame = cv2.resize(frame,(1000,900))
    cv2.imshow('Frame',frame)
        
    #if the key = 1 and q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()


In [16]:
radius

3.9052248001098633